In [1]:
import pandas as pd
import numpy as np
import yaml

In [2]:
df = pd.read_pickle("../Data/netflix-complete.pickle")

In [3]:
import requests

url = "https://movie-database-alternative.p.rapidapi.com/"

headers = {
	"X-RapidAPI-Key": "e8627fd873msh7a38a0dac6fe673p163bf7jsn57837257ac2e",
	"X-RapidAPI-Host": "movie-database-alternative.p.rapidapi.com"
}

In [4]:
def pull(row):
    imdb_ids = []
    querystring = {
        "s":row["title"],
        "r":"json",
        "type":row["vtype"],
        "y":row["year"]
        }
    try:
        response = requests.request("GET", url, headers=headers, params=querystring).json()["Search"]
        for result in response:
            if result["Title"].casefold() == row["title"].casefold() and int(result["Year"]) == row["year"] and result["Type"] == row["vtype"]:
                imdb_ids.append(result["imdbID"])
        return " | ".join(imdb_ids).strip()
    except:
        return None


In [5]:
from tqdm import tqdm
tqdm.pandas()

df["imdbid"] = df.progress_apply(lambda row: pull(row), axis=1)

100%|██████████| 17850/17850 [2:27:35<00:00,  2.02it/s]  


In [9]:
df[df["title"] == "King of Boys: The Return of the King"]

,id,title,img,vtype,nfid,synopsis,year,imdbid,clist,titledate,imdb_rating,genre,votes
5469,73580,King of Boys: The Return of the King,https://occ-0-3607-1432.1.nflxso.net/dnm/api/v...,series,81342973,Alhaja Eniola Salami starts anew and sets her ...,2021,tt15192042,"LT:Lithuania ,PL:Poland ,FR:France ,IS:Iceland...",2021-08-28,9,TV Dramas | Political TV Shows | Crime TV Show...,1860957


In [11]:
from bs4 import BeautifulSoup
def get_imdb_ratings(row):
    try:
        page = requests.get(f"https://www.imdb.com/title/{row['imdbid'].strip()}/ratings")
        soup = BeautifulSoup(page.content, "html.parser")
        return soup.find_all("span", attrs={"class":"ipl-rating-star__rating"})[0].text
    except:
        return None

In [14]:
from tqdm import tqdm
tqdm.pandas()

df["imdb_rating"] = df.progress_apply(lambda row: get_imdb_ratings(row), axis=1)

100%|██████████| 17850/17850 [2:26:26<00:00,  2.03it/s]  


In [17]:
def get_votes(row):
    imdbid = row["imdbid"]
    page = requests.get(f"https://www.imdb.com/title/{imdbid}/ratings")
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        st = soup.find_all("div", attrs={"class":"allText"})[0].text.strip()
        st_lower = st[:st.find("IMDb")].strip().replace(",", "")
        return int(st_lower)
    except:
        return 0

In [18]:
from tqdm import tqdm
tqdm.pandas()

df["votes"] = df.progress_apply(lambda row: get_votes(row), axis=1)

100%|██████████| 17850/17850 [2:45:09<00:00,  1.80it/s]  


In [19]:
df.to_pickle("new_netflix_complete.pickle")

In [21]:
df[df["title"] == "King of Boys: The Return of the King"]

,id,title,img,vtype,nfid,synopsis,year,imdbid,clist,titledate,imdb_rating,genre,votes
5469,73580,King of Boys: The Return of the King,https://occ-0-3607-1432.1.nflxso.net/dnm/api/v...,series,81342973,Alhaja Eniola Salami starts anew and sets her ...,2021,tt15192042,"LT:Lithuania ,PL:Poland ,FR:France ,IS:Iceland...",2021-08-28,6.8,TV Dramas | Political TV Shows | Crime TV Show...,286
